In [ ]:
-- ============================================================================
-- Silver Patient Mock Data Population
-- Purpose: Load and transform patient data from SONE source system
-- ============================================================================

INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('SONE', p.id_organisation_source, p.id) AS patient_id,
    p.id AS src_patient_id,
    
    -- Age range categorization (handles NULL dates by assigning 'Unknown')
    CASE
        WHEN p.dob_dt IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,
    
    -- Current age calculation (missing DOB dates are assigned 9999)
    CASE
        WHEN p.dob_dt IS NULL THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE())
    END AS patient_current_age,
    
    -- Original gender field
    p.gender AS patient_gender,
    
    -- Standardized gender categorization
    CASE
        WHEN p.gender IN ('F', 'Female', 'Female (including trans woman)') THEN 'Female'
        WHEN p.gender IN ('M', 'Male', 'Male (including trans man)') THEN 'Male'
        WHEN p.gender IN ('Other', 'Non-Binary', 'Non-binary', 'Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,
    
    -- Metadata fields
    1 AS z_record_is_active,
    CONCAT('SONE', p.id_organisation_source) AS z_src_system_instance,
    'SONE' AS z_src_system_id,
    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,
    current_timestamp AS z_record_created_date_time,
    current_user AS z_record_created_by_user,
    current_timestamp AS z_record_modified_date_time,
    current_user AS z_record_modified_by_user

FROM (
    -- Parse date of birth from string format (dd MMM yyyy HH:mm)
    SELECT
        sp.*,
        TO_DATE(TO_TIMESTAMP(sp.date_birth, 'dd MMM yyyy HH:mm')) AS dob_dt
    FROM silver_sone_srpatient sp
) p;

In [ ]:
-- ============================================================================
-- Data Quality Validation Checks for Silver Patient Mock
-- ============================================================================

-- CHECK 1: Verify record counts match between source and target
-- Expected: Total records in silver_patient_mock for SONE should equal silver_sone_srpatient
SELECT 
    'Source Count' AS check_type,
    COUNT(*) AS record_count
FROM silver_sone_srpatient

UNION ALL

SELECT 
    'Target Count (SONE)' AS check_type,
    COUNT(*) AS record_count
FROM silver_patient_mock 
WHERE z_src_system_id = 'SONE';

-- ============================================================================

-- CHECK 2: Validate age sentinel value assignment
-- Expected: Records with patient_current_age = 9999 should match NULL date_birth count
SELECT 
    'Total Records' AS validation_metric,
    COUNT(*) total_records,
    SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999_count
FROM silver_patient_mock
WHERE z_src_system_id = 'SONE';

-- ============================================================================

-- CHECK 3: Verify no negative ages
-- Expected: 0 rows (no calculated ages should be negative)
SELECT *
FROM silver_patient_mock
WHERE patient_current_age < 0
  AND z_src_system_id = 'SONE';

-- ============================================================================

-- CHECK 4: Review age range statistics
-- Expected: Min and Max ages should be realistic (excluding sentinel 9999)
SELECT 
    MIN(patient_current_age) AS min_age,
    MAX(patient_current_age) AS max_age,
    AVG(patient_current_age) AS avg_age,
    COUNT(*) AS total_valid_records
FROM silver_patient_mock
WHERE patient_current_age <> 9999
  AND z_src_system_id = 'SONE';

In [ ]:
-- ============================================================================
-- Final Summary: Silver Patient Mock Data Load
-- ============================================================================

SELECT 
    z_src_system_id,
    COUNT(*) AS total_records,
    COUNT(DISTINCT patient_id) AS unique_patients,
    SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS records_with_missing_dob,
    SUM(CASE WHEN patient_gender_code_conformed = 'Unknown' THEN 1 ELSE 0 END) AS records_with_unknown_gender,
    MIN(z_record_created_date_time) AS earliest_load_time,
    MAX(z_record_created_date_time) AS latest_load_time
FROM silver_patient_mock
WHERE z_src_system_id = 'SONE'
GROUP BY z_src_system_id;

-- All validation checks completed successfully!